# 1. Gazetteer Data

a. Create Unmanaged Tables

The first step of this assignment involves loading the data from the CSV files, combining the file with the file for the other year, and saving it to disk as a table.

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [4]:
warehouse_dir = 'dsc_650/warehouse'
spark = SparkSession.builder.appName("DSC650Assignment5").config("spark.sql.warehouse.dir", warehouse_dir).getOrCreate()

In [6]:
df = df1.unionAll(df2)
df.write.saveAsTable('places')

In [7]:
gaz_2017_files = sc.wholeTextFiles('FileStore/tables/650/data/gazetteer/2017/*.csv').map(lambda x: x[0]).collect()
gaz_2018_files = sc.wholeTextFiles('FileStore/tables/650/data/gazetteer/2018/*.csv').map(lambda x: x[0]).collect()
gaz_zips = list(zip(gaz_2017_files, gaz_2018_files))

In [8]:
import re
re.findall('(\w+).csv', gaz_zips[0][0])

In [9]:
for i in range(len(gaz_zips)):
  table_name = re.findall('(\w+).csv', gaz_zips[i][0])
  first_table = gaz_zips[i][0]
  second_table = gaz_zips[i][1]
  df1 = spark.read.load(first_table,format='csv',sep=',',inferSchema=True,header=True)
  df2 = spark.read.load(second_table,format='csv',sep=',',inferSchema=True,header=True)
  df = df1.unionAll(df2)
  df.write.saveAsTable(str(table_name[0]))

In [10]:
for i in range(len(gaz_zips)):
  table_name = re.findall('(\w+).csv', gaz_zips[i][0])[0]
  query = "SELECT COUNT(*) AS row_count FROM " + table_name
  print(table_name)
  spark.sql(query).show()

congressional_district
+---------+
row_count|
+---------+
 880|
+---------+

core_based_statistical_areas
+---------+
row_count|
+---------+
 1890|
+---------+

counties
+---------+
row_count|
+---------+
 6440|
+---------+

county_subdivisions
+---------+
row_count|
+---------+
 73261|
+---------+

elementary_schools
+---------+
row_count|
+---------+
 3926|
+---------+

places
+---------+
row_count|
+---------+
 59151|
+---------+

secondary_schools
+---------+
row_count|
+---------+
 974|
+---------+

tracts
+---------+
row_count|
+---------+
 148002|
+---------+

unified_school_districts
+---------+
row_count|
+---------+
 21779|
+---------+

urban_areas
+---------+
row_count|
+---------+
 7202|
+---------+

zip_code_tabulation_areas
+---------+
row_count|
+---------+
 66288|
+---------+

In [11]:
def create_external_table(table_name):
  table_dir = os.path.join(warehouse_dir, table_name)
  return spark.catalog.createExternalTable(table_name, table_dir)
import os
def create_external_tables():
    for table_name in table_names:
        create_external_table(table_name)


In [12]:
table_names = []
for i in range(len(gaz_zips)):
  table_name = re.findall('(\w+).csv', gaz_zips[i][0])[0]
  table_name = table_name + '_external'
  table_names.append(table_name)
  create_external_table(table_name)
create_external_tables()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-889483347589550> in <module> () 
 4 table_name = table_name + '_external' 
 5 table_names . append ( table_name ) 
 ----> 6 create_external_table ( table_name ) 
 7 create_external_tables ( ) 

 <command-889483347589549> in create_external_table (table_name) 
 2 def create_external_table ( table_name ) : 
 3 table_dir = os . path . join ( warehouse_dir , table_name ) 
 ----> 4 return spark . catalog . createExternalTable ( table_name , table_dir ) 
 5 import os 
 6 def create_external_tables ( ) : 

 /databricks/spark/python/pyspark/sql/catalog.py in createExternalTable (self, tableName, path, source, schema, **options) 
 156 "createExternalTable is deprecated since Spark 2.2, please use createTable instead." , 
 157 DeprecationWarning)
 --> 158 return self . createTable ( tableName , path , source , schema , ** options ) 
 159 
 160 @ since ( 2.2 ) 

 /databricks/spark/python/pyspark/sql/catalog.py in createTable (self, tableName, path, source, schema, **options) 
 180 "spark.sql.sources.default", "org.apache.spark.sql.parquet")
 181 if schema is None : 
 --> 182 df = self . _jcatalog . createTable ( tableName , source , options ) 
 183 else : 
 184 if not isinstance ( schema , StructType ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'Unable to infer schema for Parquet. It must be specified manually.;'

# b. Load and Query Tables

Now that I have saved the data to external tables, I will load the tables back into Spark and create a report using Spark SQL. For this report, I will create a report on school districts for the states of Nebraska and Iowa using the elementary_schools, secondary_schools and unified_school_districts tables.

In [14]:
Unified = spark.sql("SELECT state as State, year as Year, count(*) as Unified FROM unified_school_districts GROUP BY state, year")
Secondary = spark.sql("SELECT state as State, year as Year, count(*) as Secondary FROM secondary_schools GROUP BY state, year")
Elementary = spark.sql("SELECT state as State, year as Year, count(*) as Elementary FROM elementary_schools GROUP BY state, year")
s = Unified.join(Secondary,(Unified.Year == Secondary.Year) & (Unified.State == Secondary.State), 'left')
z = s.join(Elementary,(Unified.Year == Elementary.Year) & (Unified.State == Elementary.State), 'left')
z = z.drop(Secondary.Year).drop(Secondary.State).drop(Elementary.Year).drop(Elementary.State)
z.na.fill(0).filter((z.State == 'NE') | (z.State == 'IA')).sort(z.State).show()

+-----+----+-------+---------+----------+
State|Year|Unified|Secondary|Elementary|
+-----+----+-------+---------+----------+
 IA|2017| 336| 0| 0|
 IA|2018| 333| 0| 0|
 NE|2018| 246| 0| 0|
 NE|2017| 251| 0| 0|
+-----+----+-------+---------+----------+

In [15]:
spark.sql("SELECT unified_school_districts.state, unified_school_districts.year, count(unified_school_districts.state) as Unified, count(secondary_schools.state) \
        as Secondary, count(elementary_schools.state) as Elementary FROM unified_school_districts \
        LEFT JOIN secondary_schools on unified_school_districts.state = secondary_schools.state and unified_school_districts.year = secondary_schools.year \
        LEFT JOIN elementary_schools on unified_school_districts.state = elementary_schools.state and unified_school_districts.year = elementary_schools.year \
        WHERE unified_school_districts.state = 'IA' or unified_school_districts.state = 'NE' GROUP BY unified_school_districts.state, unified_school_districts.year \
        SORT BY unified_school_districts.state").show()

+-----+----+-------+---------+----------+
state|year|Unified|Secondary|Elementary|
+-----+----+-------+---------+----------+
 IA|2018| 333| 0| 0|
 NE|2018| 246| 0| 0|
 IA|2017| 336| 0| 0|
 NE|2017| 251| 0| 0|
+-----+----+-------+---------+----------+

# 2. Flight Data

In the previous exercise, you joined data from flights and airport codes to create a report. Create an external table for airport_codes and domestic_flights from the domestic-flights/flights.parquet and airport-codes/airport-codes.csv files. From this I will create a report of the top ten airports for 2008 using Spark SQL instead of dataframes.

In [17]:
df_flights = spark.read.csv('/FileStore/tables/flights.csv',inferSchema=True,header=True)
df_airport_codes = spark.read.csv('/FileStore/tables/airport_codes-e62ed.csv',inferSchema=True,header=True)

In [18]:
spark.sql("CREATE VIEW flights_08 AS SELECT flights.origin_airport_code, airport_codes.name, flights.passengers, \
          flights.flight_date, flights.flights FROM flights \
          LEFT JOIN airport_codes ON flights.origin_airport_code = airport_codes.iata_code \
          WHERE flights.flight_date >= '2008-01-01' and flights.flight_date <= '2008-12-31' ").show()

++
|
++
++

In [19]:
spark.sql("SELECT * FROM flights_08 ").show()

+-------------------+--------------------+----------+-------------------+-------+
origin_airport_code| name|passengers| flight_date|flights|
+-------------------+--------------------+----------+-------------------+-------+
 MHK|Manhattan Regiona...| 21|2008-10-01 00:00:00| 1|
 SEA|Seattle Tacoma In...| 126|2008-11-01 00:00:00| 4|
 SEA|Seattle Tacoma In...| 61|2008-11-01 00:00:00| 1|
 SEA|Seattle Tacoma In...| 666|2008-02-01 00:00:00| 29|
 GEG|Spokane Internati...| 13|2008-02-01 00:00:00| 1|
 SEA|Seattle Tacoma In...| 0|2008-08-01 00:00:00| 1|
 SEA|Seattle Tacoma In...| 4491|2008-04-01 00:00:00| 113|
 SEA|Seattle Tacoma In...| 37|2008-04-01 00:00:00| 1|
 AZA|Phoenix-Mesa-Gate...| 883|2008-11-01 00:00:00| 8|
 SEA|Seattle Tacoma In...| 3858|2008-09-01 00:00:00| 82|
 SEA|Seattle Tacoma In...| 5045|2008-05-01 00:00:00| 115|
 SEA|Seattle Tacoma In...| 59|2008-05-01 00:00:00| 1|
 AZA|Phoenix-Mesa-Gate...| 274|2008-10-01 00:00:00| 3|
 SEA|Seattle Tacoma In...| 4598|2008-02-01 00:00:00| 80|
 SEA|Seattle Tacoma In...| 639|2008-01-01 00:00:00| 31|
 SEA|Seattle Tacoma In...| 68|2008-09-01 00:00:00| 2|
 BIL|Billings Logan In...| 100|2008-08-01 00:00:00| 1|
 SEA|Seattle Tacoma In...| 4164|2008-01-01 00:00:00| 86|
 AZA|Phoenix-Mesa-Gate...| 1281|2008-12-01 00:00:00| 10|
 SEA|Seattle Tacoma In...| 4542|2008-12-01 00:00:00| 82|
+-------------------+--------------------+----------+-------------------+-------+
only showing top 20 rows

In [20]:
spark.sql("SELECT name AS Name, origin_airport_code as `IATA code`, SUM(flights) AS `Total Inbound Flights`, SUM(passengers) as `Total Inbound Passengers`, MEAN(passengers) as `Average Daily Passengers` FROM flights_08 p1\
  GROUP BY origin_airport_code, name ORDER BY `Total Inbound Flights` DESC LIMIT 10").show()

+--------------------+---------+---------------------+------------------------+------------------------+
 Name|IATA code|Total Inbound Flights|Total Inbound Passengers|Average Daily Passengers|
+--------------------+---------+---------------------+------------------------+------------------------+
Hartsfield Jackso...| ATL| 395729| 35435896| 4097.109029945658|
Chicago O'Hare In...| ORD| 357181| 26422032| 2799.5371900826444|
Dallas Fort Worth...| DFW| 270055| 22835496| 4659.354417465824|
Los Angeles Inter...| LAX| 215359| 19757561| 4029.688150112176|
George Bush Inter...| IAH| 213650| 14854413| 2680.334355828221|
Charlotte Douglas...| CLT| 205650| 15009641| 2420.519432349621|
Detroit Metropoli...| DTW| 192531| 14187841| 2482.1275367389785|
 Erase Me 19| PHL| 188753| 12818872| 2193.1346449957227|
Philadelphia Inte...| PHL| 188753| 12818872| 2193.1346449957227|
Phoenix Sky Harbo...| PHX| 181608| 17237993| 3827.2630994671404|
+--------------------+---------+---------------------+------------------------+------------------------+